In [42]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch.optim as optim
from torch_geometric.data import Data, Batch
import torch.nn.functional as F


In [43]:
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Using Apple Silicon GPU")
elif torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using CUDA GPU")
else:
    device = torch.device('cpu')

Using Apple Silicon GPU


In [44]:
def getData(path):
    train_file = np.load(path+"/train.npz")
    train_data = train_file['data']
    test_file = np.load(path+"/test_input.npz")
    test_data = test_file['data']
    print(f"Training Data's shape is {train_data.shape} and Test Data's is {test_data.shape}")
    return train_data, test_data

In [45]:
trainData, testData = getData("./data/")

Training Data's shape is (10000, 50, 110, 6) and Test Data's is (2100, 50, 50, 6)


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class WindowedNormalizedDataset(Dataset):
    def __init__(self, data, window_size, forecast_horizon):
        self.data = data
        self.window_size = window_size
        self.forecast_horizon = forecast_horizon

        self.indices = []
        for sample in range(data.shape[0]):
            for t in range(data.shape[2] - window_size - forecast_horizon + 1):
                self.indices.append((sample, t))

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        sample_idx, t = self.indices[idx]

        origin = self.data[sample_idx, 0, 49, :2].copy()
        x = self.data[sample_idx, :, t:t+self.window_size, :]
        x[..., :2] -= origin

        x[..., :4] = x[..., :4] / 10
        y = self.data[sample_idx, 0, t+self.window_size:t+self.window_size+self.forecast_horizon, :2]
        y = y/10
        # print(x.shape, y.shape)

        return (
        torch.tensor(x, dtype=torch.float32),
        torch.tensor(y, dtype=torch.float32),
        torch.tensor(origin, dtype=torch.float32)  # normalize origin for consistency
        )

In [6]:
torch.manual_seed(42)
# np.random.seed(42)

scale = 9.0

N = trainData.shape[0]
val_size = int(0.1 * N)
train_size = N - val_size

train_dataset = WindowedNormalizedDataset(data = trainData[:train_size], window_size=50, forecast_horizon=60)
validation_dataset = WindowedNormalizedDataset(data = trainData[train_size:], window_size=50, forecast_horizon=60)


In [7]:
class EncoderDecoderModel(nn.Module):
    def __init__(self, infeatures, outfeatures=2):
        super().__init__()
        # Encoder
        self.layer1 = nn.Linear(infeatures, 32)
        self.layer2 = nn.Linear(32, 64)
        self.layer3 = nn.Linear(64, 128)
        self.encoderlstm = nn.LSTM(128, 256, num_layers=2, batch_first=True, dropout=0.3)
        
        # Changed pooling target from 20 to 60
        self.pool = nn.AdaptiveAvgPool1d(60)
        self.dropout = nn.Dropout(0.2)
        
        # Decoder
        self.decoderlstm = nn.LSTM(256, 128, num_layers=2, batch_first=True, dropout=0.3)
        self.layer10 = nn.Linear(128, 64)
        self.layer11 = nn.Linear(64, 32)
        self.layer12 = nn.Linear(32, outfeatures)
        
        # Skip connections
        self.skip1 = nn.Linear(32, 32)
        self.skip2 = nn.Linear(64, 64)
        self.skip3 = nn.Linear(128, 128)
        self.skip4 = nn.Linear(256, 256)

    def forward(self, x):
        batch_size, channels, height, width = x.shape
        
        # Encoder
        out1 = nn.ReLU()(self.layer1(x))
        out2 = nn.ReLU()(self.layer2(out1))
        out3 = nn.ReLU()(self.layer3(out2))
        
        # LSTM processing
        tempout3 = out3.view(batch_size, -1, out3.size(-1))
        out4, _ = self.encoderlstm(tempout3)
        
        # Changed pooling to 60
        tempout4 = self.pool(out4.permute(0, 2, 1))
        tempout4 = tempout4.permute(0, 2, 1)
        lstmskip = tempout4 + self.skip4(tempout4)
        
        # Decoder LSTM
        out5, _ = self.decoderlstm(lstmskip)
        
        out3_reduced = F.adaptive_avg_pool2d(out3.permute(0, 3, 1, 2), (60, 1)).squeeze(-1).permute(0, 2, 1)
        mlpskip1 = out3_reduced + self.skip3(out5)
        out6 = nn.ReLU()(self.layer10(mlpskip1))
        
        out2_reduced = F.adaptive_avg_pool2d(out2.permute(0, 3, 1, 2), (60, 1)).squeeze(-1).permute(0, 2, 1)
        mlpskip2 = out2_reduced + self.skip2(out6)
        out7 = nn.ReLU()(self.layer11(mlpskip2))
        
        out1_reduced = F.adaptive_avg_pool2d(out1.permute(0, 3, 1, 2), (60, 1)).squeeze(-1).permute(0, 2, 1)
        mlpskip3 = out1_reduced + self.skip1(out7)
        out8 = self.layer12(mlpskip3)
        
        return out8

# Verify the output shape
def xavier_init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_normal_(m.weight)
        if m.bias is not None:
            torch.nn.init.constant_(m.bias, 0)
        
model = EncoderDecoderModel(6, 2)
model.apply(xavier_init_weights)

# test = torch.randn(128, 50, 50, 6)
# out = model(test)
# print(out.shape)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")
model.to(device)

Total parameters: 1359906


EncoderDecoderModel(
  (layer1): Linear(in_features=6, out_features=32, bias=True)
  (layer2): Linear(in_features=32, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=128, bias=True)
  (encoderlstm): LSTM(128, 256, num_layers=2, batch_first=True, dropout=0.3)
  (pool): AdaptiveAvgPool1d(output_size=60)
  (dropout): Dropout(p=0.2, inplace=False)
  (decoderlstm): LSTM(256, 128, num_layers=2, batch_first=True, dropout=0.3)
  (layer10): Linear(in_features=128, out_features=64, bias=True)
  (layer11): Linear(in_features=64, out_features=32, bias=True)
  (layer12): Linear(in_features=32, out_features=2, bias=True)
  (skip1): Linear(in_features=32, out_features=32, bias=True)
  (skip2): Linear(in_features=64, out_features=64, bias=True)
  (skip3): Linear(in_features=128, out_features=128, bias=True)
  (skip4): Linear(in_features=256, out_features=256, bias=True)
)

In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EncoderDecoderModel(nn.Module):
    def __init__(self, infeatures = 6, outfeatures=2, agents = 50, timestamp = 50, windowSize = 60, ):
        super().__init__()
        
        # batch_size, x_dim, y_dim, features = x.shape

        # x = x.view(batch_size, x_dim * y_dim, features)
        self.project_layer = nn.Linear(agents * timestamp, 60)  
        # MLP Encoder
        self.layer1 = nn.Linear(infeatures, 32)
        self.layer2 = nn.Linear(32, 64)
        self.layer3 = nn.Linear(64, 128)
        
        # LSTM Encoder (2 layers)
        self.encoder_lstm = nn.LSTM(128, 256, num_layers=2, batch_first=True)
        
        # Decoder LSTM (1 layer)
        self.decoder_lstm = nn.LSTM(256, 128, num_layers=1, batch_first=True)
        
        # MLP Decoder
        self.layer10 = nn.Linear(128, 64)
        self.layer11 = nn.Linear(64, 32)
        self.layer12 = nn.Linear(32, outfeatures)
        

    def forward(self, x):
        # x: [batch_size, seq_len, infeatures]
        batch_size, x_dim, y_dim, features = x.shape

        x = x.view(batch_size, x_dim * y_dim, features)
        # project = self.project_layer(x_dim * y_dim, 60)  # maps x*y → 60
        x = x.permute(0, 2, 1)  # (batch, features, x*y)
        x = self.project_layer(x)          # (batch, features, 60)
        x = x.permute(0, 2, 1)

        x = F.relu(self.layer1(x))
        
        
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer3(x))
        
        # # LSTM Encoder
        # print(out.shape)
        x, _ = self.encoder_lstm(x)
        # print(x.shape)
        
        # # LSTM Decoder
        x, _ = self.decoder_lstm(x)
        # print(x.shape)
        
        # # Decoder MLP
        x = F.relu(self.layer10(x))
        x = F.relu(self.layer11(x))
        x = self.layer12(x)
        
        return x

# Weight initialization
def xavier_init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_normal_(m.weight)
        if m.bias is not None:
            torch.nn.init.constant_(m.bias, 0)

model = EncoderDecoderModel(6, 2)
model.apply(xavier_init_weights)

test = torch.randn(128, 50, 50, 6)
out = model(test)
print(out.shape)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")
model.to(device)


torch.Size([128, 60, 2])
Total parameters: 1290350


EncoderDecoderModel(
  (project_layer): Linear(in_features=2500, out_features=60, bias=True)
  (layer1): Linear(in_features=6, out_features=32, bias=True)
  (layer2): Linear(in_features=32, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=128, bias=True)
  (encoder_lstm): LSTM(128, 256, num_layers=2, batch_first=True)
  (decoder_lstm): LSTM(256, 128, batch_first=True)
  (layer10): Linear(in_features=128, out_features=64, bias=True)
  (layer11): Linear(in_features=64, out_features=32, bias=True)
  (layer12): Linear(in_features=32, out_features=2, bias=True)
)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EncoderDecoderModel(nn.Module):
    def __init__(self, infeatures=6, outfeatures=2):
        super().__init__()

        self.conv1d = nn.Conv1d(in_channels=infeatures, out_channels=64, kernel_size=3, stride=42, padding=1)

        self.layer1 = nn.Linear(64, 128)
        self.layer2 = nn.Linear(128, 256)

        self.encoder_lstm = nn.LSTM(256, 256, num_layers=2, batch_first=True)

        self.decoder_lstm = nn.LSTM(256, 256, num_layers=2, batch_first=True)

        self.layer10 = nn.Linear(256, 128)
        self.layer11 = nn.Linear(128, 64)
        self.layer12 = nn.Linear(64, 32)
        self.layer13 = nn.Linear(32, 16)
        self.layer14 = nn.Linear(16, outfeatures)

        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        batch_size, x_dim, y_dim, features = x.shape

        x = x.permute(0, 3, 1, 2).reshape(batch_size, features, -1)  # (batch, infeatures, 2500)

        x = self.conv1d(x)  # (batch, 64, 60)
        x = x.permute(0, 2, 1)  # (batch, 60, 64)

        x = F.relu(self.layer1(x))
        x = self.dropout(x)
        x = F.relu(self.layer2(x))
        x = self.dropout(x)

        x, _ = self.encoder_lstm(x)
        x, _ = self.decoder_lstm(x)

        x = F.relu(self.layer10(x))
        x = F.relu(self.layer11(x))
        x = F.relu(self.layer12(x))
        x = F.relu(self.layer13(x))
        x = self.layer14(x)

        return x


model = EncoderDecoderModel(6, 2)
model.apply(xavier_init_weights)

test = torch.randn(128, 50, 50, 6)
out = model(test)
print(out.shape)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")
model.to(device)


torch.Size([128, 60, 2])
Total parameters: 2191698


EncoderDecoderModel(
  (conv1d): Conv1d(6, 64, kernel_size=(3,), stride=(42,), padding=(1,))
  (layer1): Linear(in_features=64, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=256, bias=True)
  (encoder_lstm): LSTM(256, 256, num_layers=2, batch_first=True)
  (decoder_lstm): LSTM(256, 256, num_layers=2, batch_first=True)
  (layer10): Linear(in_features=256, out_features=128, bias=True)
  (layer11): Linear(in_features=128, out_features=64, bias=True)
  (layer12): Linear(in_features=64, out_features=32, bias=True)
  (layer13): Linear(in_features=32, out_features=16, bias=True)
  (layer14): Linear(in_features=16, out_features=2, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

In [41]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# torch.cuda.empty_cache()
# print(device)
# model.load_state_dict(torch.load("./models/modelF/medium_model_0.0062230645.pth"))  

trainDataLoader = DataLoader(train_dataset, batch_size=128, shuffle=True)
testDataLoader = DataLoader(validation_dataset, batch_size=128)
model.to(device)
# Training setup
epochs = 1000
lossFn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

tLoss = 10000
vLoss = 10000
# Iused 0.0008, 0.0001

for each_epoch in range(epochs):
    model.train()
    runningLoss = 0.0
    loop = tqdm(trainDataLoader, desc=f"Epoch [{each_epoch+1}/{epochs}]")
    totalSamples = 0
    for batchX, batchY, _ in loop:
        batchX, batchY = batchX.to(device, non_blocking=True), batchY.to(device, non_blocking=True)
        output = model(batchX)
        # print("Pred Training Shape: ", output.shape,"True Value", batchY.shape)
        # break
        loss = lossFn(output, batchY)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        runningLoss += loss.item()
        totalSamples += batchX.size(0)

    avgLoss = runningLoss/len(trainDataLoader)
    # print(avgLoss)#, runningLoss, len(trainDataLoader.dataset))    
    # break

    model.eval()
    with torch.no_grad():
        testloss = 0.0
        unnorm_loss = 0.0
        for testX, testY, origin in testDataLoader:
            testX, testY, origin = testX.to(device), testY.to(device), origin.to(device)
    
            pred = model(testX)
            # print(pred.shape, testY.shape, origin.shape)
            origin = origin.unsqueeze(1).expand(-1, 60, -1)
            # print(pred+origin)
            # break
            loss = lossFn(pred, testY)
            pred_absolute = (pred * 10) + origin 
            true_unnorm = (testY * 10) + origin
    
            testloss += loss.item()

            unnorm_loss += nn.MSELoss()(pred_absolute, true_unnorm).item()
        
        avgtestloss = testloss / len(testDataLoader)
        avgUnnormLoss = unnorm_loss /len(testDataLoader)

    # break
    print(f"Epoch {each_epoch + 1}, Training Loss: {avgLoss:},  Validation Loss:{avgtestloss:} , Absolute MSE:{avgUnnormLoss}")
    # break
    if(avgLoss < tLoss and avgtestloss < vLoss):
        tLoss = avgLoss
        vLoss = avgtestloss
        torch.save(model.state_dict(), f'./models/modelL/medium_model_{avgLoss:.10f}.pth')

    with open("./models/modelL/log_loss.txt", 'a') as f:
        f.write(f"{each_epoch + 1},{avgLoss:.10f},{avgtestloss:.10f},\n")
        
    torch.cuda.empty_cache()
    scheduler.step()


Epoch [1/1000]:   0%|          | 0/71 [00:00<?, ?it/s]

Epoch [1/1000]: 100%|██████████| 71/71 [00:20<00:00,  3.53it/s]


Epoch 1, Training Loss: 113979.82394366198,  Validation Loss:107880.9609375 , Absolute MSE:10788096.375


Epoch [2/1000]: 100%|██████████| 71/71 [00:18<00:00,  3.94it/s]


Epoch 2, Training Loss: 108308.75704225352,  Validation Loss:100067.1298828125 , Absolute MSE:10006713.25


...

Epoch 86, Training Loss: 72095.05633802817,  Validation Loss:67050.1083984375 , Absolute MSE:6705010.6875


Epoch [87/1000]: 100%|██████████| 71/71 [00:17<00:00,  3.98it/s]


Epoch 87, Training Loss: 72024.59848151408,  Validation Loss:67051.96044921875 , Absolute MSE:6705196.125


Epoch [88/1000]:  21%|██        | 15/71 [00:04<00:16,  3.45it/s]


KeyboardInterrupt: 

In [47]:
# A = torch.randn(32, 60, 2)  
# B = torch.randn(32, 2)      
# print(B)
# # Expand B to shape (32, 60, 2)
# B_expanded = B.unsqueeze(1).expand(-1, 10, -1)
# B_expanded